In [1]:
import sys 
import pandas as pd
import logging

sys.path.append('../')

import utils.llm 
# import utils.scoring
from typing import Literal

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

class Config:
    DATA_DIR = '../construct_dataset/'
    DATA_OUT_DIR = '../data_out/'
    TEMPERATURE = 0.01 # responseを得るのを利用したtemperatureと同じ値に設定
    SAMPLE_N = 30
    RANDOM_STR_LENGTH =  80
    CHAR_KIND : Literal['hiragana','katakana','kanji'] = 'katakana'
    TARGET_QUIZ_RANGE = list(range(0, 201))
    
def build_prompt(query:str):
    
    import random
    
    # ランダム文字列を生成
    RANDOM_STR_LENGTH = Config.RANDOM_STR_LENGTH
    
    hiragana_list = [chr(i) for i in range(ord('ぁ'), ord('ん') + 1)]
    katakana_list = [chr(i) for i in range(ord('ァ'), ord('ン') + 1)]
    
    random_str = ''
    
    for i in range(RANDOM_STR_LENGTH):
        if Config.CHAR_KIND == 'hiragana':
            random_str += random.choice(hiragana_list)
        elif Config.CHAR_KIND == 'katakana':
            random_str += random.choice(katakana_list)
        elif Config.CHAR_KIND == 'kanji':
            raise NotImplementedError
        else:
            raise ValueError('Config.CHAR_KIND must be hiragana or katakana or kanji')
    
    
    
    prompt = f'{random_str} {query}'
    
    return prompt
    


quiz_df = pd.read_csv(Config.DATA_DIR + 'df_JAQKET_qa_annot1.csv')

num_questions = quiz_df.shape[0]

# num_questions = 2


In [2]:
llm = utils.llm.LLM()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [3]:
def sampling(query:str,temperature:float,sample_n:int) -> list[str]:
    
    sampled = []
    
    for i in range(sample_n):
        
        q = build_prompt(query)
        sampled.append(llm.generate(q,temperature))
    
    logger.info("sampled: {}".format(sampled))
    
    return sampled


In [4]:
# serial_number, qid, query, responseを記録するdf

df = pd.DataFrame(columns=['serial_number','qid','query','sample'])

serial_number = 0

for i in Config.TARGET_QUIZ_RANGE:
    
    logger.info("processing {} th question".format(i))
    
    qid = quiz_df.iloc[i]['qid']
    query = quiz_df.iloc[i]['question']
    response = quiz_df.iloc[i]['generated_answer']
    
    sampled = sampling(query,Config.TEMPERATURE,Config.SAMPLE_N)
    
    for sample in sampled:
        
        #  The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        
        df = pd.concat([df,pd.DataFrame([[serial_number,qid,query,sample]],columns=['serial_number','qid','query','sample'])])
        
        serial_number += 1


processing 0 th question
sampled: ['マンタ', 'マンタ', 'マングース', 'ウミガメ', 'ウミガメ', 'マンタ', 'ザノオヰアトトウレパムゼポァノデェァクギザオペゾダックナシキヂゴセデネノテサミベミホベオキピバカケガゼポフキホタロプナァヱンテエワマザッョサパギォゲモハゥヅパン和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる動物は、ザノオヰアトトウレパムゼポァノデェァクギザオペゾダックナシキヂゴ', 'マンタ', 'ウミガメ', 'マンタ', 'マンタ', 'マングース', 'マングース', 'マンタ', 'マングース', '貝殻のない巻貝', 'マンドリル', 'マンタ', 'ウミガメ', 'ウミガメ', 'マングース', 'マンジュウイシモエガニ', 'ウミガメ', 'マンタ', 'マングース', 'マンタ', 'マンタ', '貝殻のない巻貝', 'マンジュウガニ', 'マンジュウガニ']
processing 1 th question
sampled: ['東方見聞録 作者はマルコ・ポーロ。彼は中国や日本を訪れた際に多くの資料を集め、それらをまとめ上げた。この作品は、マルコ・ポーロによって書かれた旅行記であり、東洋の文化や歴史について広く書かれている。', '東方見聞録 マルコ・ポーロは、1453年から1547年まで中国に滞在し、多くの著作や絵画を残しました。彼は、東方見聞録の中で、ヴェネツィアやフィレンツェなどのイタリアの都市や、日本や朝鮮など東アジアの国々についての記述を残しています。この本は、東洋と西洋の文化交流の重要な記録として高く評価されています。', '東方見聞録 マルコ・ポーロは、1453年3月26日にヴェネツィアに到着し、4月8日にヴェネツィアを発って帰国しました。彼は、東方見聞録の中で、ヴェネツィアやイタリア半島の様々な地域について語っています。マルコ・ポーロは、ヴェネツィアとイタリア半島を広く旅し、多くの美しい景色や文化的な伝統を発見しました。マルコ・ポーロは、ヴェネツィアやイタリア半島の様々な地域について、東方見聞録の中で非常に印象的に描いています。', '東方見聞録 マルコ・ポーロは、1453年から1547年まで中国に滞在し、多くの著作や絵画を残

In [5]:
df.shape

(6030, 4)

In [6]:
import datetime

info = 'temperature_{}_sample_n_{}_random_str_length_{}'.format(Config.TEMPERATURE,Config.SAMPLE_N,Config.RANDOM_STR_LENGTH)
suffix = datetime.datetime.now().strftime('%m_%d_%H_%M_%S')
char_kind = Config.CHAR_KIND

df.to_csv(Config.DATA_OUT_DIR + 'random_query_{}_{}_{}.csv'.format(info,char_kind,suffix),index=False)




In [7]:
df

,serial_number,qid,query,sample
0,0,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,マンタ
0,1,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,マンタ
0,2,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,マングース
0,3,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,ウミガメ
0,4,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,ウミガメ
...,...,...,...,...
0,6025,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,ツゲヨバドソペヒヅバヮテョタミグォコトボゥグボトモベクエヌジグニャギウマョタロオヅレザテドチ...
0,6026,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦類
0,6027,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦類
0,6028,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦類
